# Out-of-State-Contributions: National Analysis

How much out-of-state money have candidates nationally raised in the 2018 election cycle, in absolute and proportional terms, thus far and how does that compare with this point in the 2014 and 2010 cycles?

In [1]:
import numpy as np
import pandas as pd

%load_ext jupyternotify

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 500)
pd.options.display.float_format = "{:,.2f}".format # Format floats

<IPython.core.display.Javascript object>

Import contributions data.

In [2]:
%%notify
contributions = pd.read_csv("data/contributions.csv")
contributions.info()

/home/jyerardi/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (14,17,18,19,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6867306 entries, 0 to 6867305
Data columns (total 23 columns):
candidate                 object
candidate_id              int64
year                      int64
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      object
contributor_street        object
contributor_city          object
contributor_state         object
contributor_zip           float64
in_out_state              object
no_veto                   object
office                    object
latest_month              object
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: float64(2), int64(2), object(19)
memory usage: 1.2+ GB


<IPython.core.display.Javascript object>

Convert the candidate ID and year columns to integer data type, the contribution date and latest month columns to datetime data type and the contributor zip column to object data type.

In [3]:
contributions["candidate_id"] = pd.to_numeric(contributions["candidate_id"], errors="coerce", downcast="integer")
contributions["year"] = pd.to_numeric(contributions["year"], errors="coerce", downcast="integer")
contributions["date"] = pd.to_datetime(contributions["date"], errors="coerce")
contributions["latest_month"] = pd.to_datetime(contributions["latest_month"], errors="coerce")
contributions["contributor_zip"] = contributions["contributor_zip"].astype(object)
contributions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6867306 entries, 0 to 6867305
Data columns (total 23 columns):
candidate                 object
candidate_id              int32
year                      int16
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      datetime64[ns]
contributor_street        object
contributor_city          object
contributor_state         object
contributor_zip           object
in_out_state              object
no_veto                   object
office                    object
latest_month              datetime64[ns]
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: datetime64[ns](2), float64(1), int16(1), int32(1), object(18)
memory usage: 1.1+ GB


Import full cycle contributions data.

In [4]:
%%notify
contributions_full_cycles = pd.read_csv("data/contributions_full_cycles.csv")
contributions_full_cycles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9106128 entries, 0 to 9106127
Data columns (total 17 columns):
candidate_id           int64
candidate              object
election_status        object
party                  object
state                  object
year                   int64
office                 object
contributor            object
amount                 float64
date                   object
contributor_street     object
contributor_city       object
contributor_state      object
contributor_zip        float64
in_out_state           object
standardized_office    object
standardized_status    object
dtypes: float64(2), int64(2), object(13)
memory usage: 1.2+ GB


<IPython.core.display.Javascript object>

Convert the candidate ID and year columns to integer data type, the contribution date column to datetime data type and the contributor zip column to object data type.

In [5]:
contributions_full_cycles["candidate_id"] = pd.to_numeric(contributions_full_cycles["candidate_id"], errors="coerce", downcast="integer")
contributions_full_cycles["year"] = pd.to_numeric(contributions_full_cycles["year"], errors="coerce", downcast="integer")
contributions_full_cycles["date"] = pd.to_datetime(contributions_full_cycles["date"], errors="coerce")
contributions_full_cycles["contributor_zip"] = contributions_full_cycles["contributor_zip"].astype(object)
contributions_full_cycles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9106128 entries, 0 to 9106127
Data columns (total 17 columns):
candidate_id           int32
candidate              object
election_status        object
party                  object
state                  object
year                   int16
office                 object
contributor            object
amount                 float64
date                   datetime64[ns]
contributor_street     object
contributor_city       object
contributor_state      object
contributor_zip        object
in_out_state           object
standardized_office    object
standardized_status    object
dtypes: datetime64[ns](1), float64(1), int16(1), int32(1), object(13)
memory usage: 1.1+ GB


## Calculate out-of-state contributions by party and year

Group by year and party and in-vs.-out-of-state contribution status and sum contributions.

In [6]:
contributions_by_party = contributions.groupby(["year", "party", "in_out_state"])["amount"].sum().reset_index()
contributions_by_party.head()

,year,party,in_out_state,amount
0,2010,Democratic,in-state,"511,599,401.37"
1,2010,Democratic,out-of-state,"63,773,103.10"
2,2010,Democratic,unknown,"1,670,811.78"
3,2010,Nonpartisan,in-state,"994,909.24"
4,2010,Nonpartisan,out-of-state,"142,553.70"


Pivot dataframe to aggregate each party's data in a single row.

In [7]:
contributions_by_party = pd.pivot_table(contributions_by_party, index=["party"], columns=["year", "in_out_state"]).reset_index()
contributions_by_party

party         amount                             \
year                                2010                              
in_out_state                    in-state  out-of-state      unknown   
0              Democratic 511,599,401.37 63,773,103.10 1,670,811.78   
1             Nonpartisan     994,909.24    142,553.70   -19,438.72   
2              Republican 701,750,841.97 57,352,634.70   875,940.46   
3             Third-Party   6,863,698.57    852,177.89 1,538,978.63   

                                                                        \
year                   2014                                       2018   
in_out_state       in-state  out-of-state       unknown       in-state   
0            438,568,693.79 74,661,507.08  6,296,267.27 694,938,481.84   
1              1,781,565.01    178,003.05      4,068.00   1,642,246.38   
2            498,905,779.94 78,945,614.02 11,198,512.65 773,921,813.64   
3             12,669,293.15    627,085.94     40,286.04   3,309,432.13   

                                           
year                                       
in_out_state   out-of-state       unknown  
0            100,883,207.06 10,117,328.46  
1                336,165.87     -2,307.58  
2             71,364,530.01  9,472,896.23  
3                339,879.37    486,249.28

Some records have no contributions for certain categories. Let's set those values equal to zero to be sure any calculations we run on them are correct.

In [8]:
contributions_by_party.fillna(0, inplace=True)

Flatten the resulting dataframe's multi-index columns.

In [9]:
contributions_by_party.columns = ["party", "10_in_state", "10_out_of_state", "10_unknown",
                                  "14_in_state", "14_out_of_state", "14_unknown",
                                  "18_in_state", "18_out_of_state", "18_unknown"
                                  ]
contributions_by_party

,party,10_in_state,10_out_of_state,10_unknown,14_in_state,14_out_of_state,14_unknown,18_in_state,18_out_of_state,18_unknown
0,Democratic,"511,599,401.37","63,773,103.10","1,670,811.78","438,568,693.79","74,661,507.08","6,296,267.27","694,938,481.84","100,883,207.06","10,117,328.46"
1,Nonpartisan,"994,909.24","142,553.70","-19,438.72","1,781,565.01","178,003.05","4,068.00","1,642,246.38","336,165.87","-2,307.58"
2,Republican,"701,750,841.97","57,352,634.70","875,940.46","498,905,779.94","78,945,614.02","11,198,512.65","773,921,813.64","71,364,530.01","9,472,896.23"
3,Third-Party,"6,863,698.57","852,177.89","1,538,978.63","12,669,293.15","627,085.94","40,286.04","3,309,432.13","339,879.37","486,249.28"


Calculate the total contributions by cycle.

In [10]:
contributions_by_party["18_total"] = contributions_by_party["18_in_state"] + contributions_by_party["18_out_of_state"] + contributions_by_party["18_unknown"]
contributions_by_party["14_total"] = contributions_by_party["14_in_state"] + contributions_by_party["14_out_of_state"] + contributions_by_party["14_unknown"]
contributions_by_party["10_total"] = contributions_by_party["10_in_state"] + contributions_by_party["10_out_of_state"] + contributions_by_party["10_unknown"]
contributions_by_party = contributions_by_party[["party", "18_in_state", "18_out_of_state", "18_unknown", "18_total", "14_in_state", "14_out_of_state", "14_unknown", "14_total", "10_in_state", "10_out_of_state", "10_unknown", "10_total"]]

Calculate the proportion of in-state, out-of-state and unknown contributions by cycle.

In [11]:
contributions_by_party["pct_18_in_state"] = contributions_by_party["18_in_state"] / (contributions_by_party["18_in_state"] + contributions_by_party["18_out_of_state"] + contributions_by_party["18_unknown"])
contributions_by_party["pct_18_out_of_state"] = contributions_by_party["18_out_of_state"] / (contributions_by_party["18_in_state"] + contributions_by_party["18_out_of_state"] + contributions_by_party["18_unknown"])
contributions_by_party["pct_18_unknown"] = contributions_by_party["18_unknown"] / (contributions_by_party["18_in_state"] + contributions_by_party["18_out_of_state"] + contributions_by_party["18_unknown"])
contributions_by_party["pct_14_in_state"] = contributions_by_party["14_in_state"] / (contributions_by_party["14_in_state"] + contributions_by_party["14_out_of_state"] + contributions_by_party["14_unknown"])
contributions_by_party["pct_14_out_of_state"] = contributions_by_party["14_out_of_state"] / (contributions_by_party["14_in_state"] + contributions_by_party["14_out_of_state"] + contributions_by_party["14_unknown"])
contributions_by_party["pct_14_unknown"] = contributions_by_party["14_unknown"] / (contributions_by_party["14_in_state"] + contributions_by_party["14_out_of_state"] + contributions_by_party["14_unknown"])
contributions_by_party["pct_10_in_state"] = contributions_by_party["10_in_state"] / (contributions_by_party["10_in_state"] + contributions_by_party["10_out_of_state"] + contributions_by_party["10_unknown"])
contributions_by_party["pct_10_out_of_state"] = contributions_by_party["10_out_of_state"] / (contributions_by_party["10_in_state"] + contributions_by_party["10_out_of_state"] + contributions_by_party["10_unknown"])
contributions_by_party["pct_10_unknown"] = contributions_by_party["10_unknown"] / (contributions_by_party["10_in_state"] + contributions_by_party["10_out_of_state"] + contributions_by_party["10_unknown"])
contributions_by_party

/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

,party,18_in_state,18_out_of_state,18_unknown,18_total,14_in_state,14_out_of_state,14_unknown,14_total,10_in_state,10_out_of_state,10_unknown,10_total,pct_18_in_state,pct_18_out_of_state,pct_18_unknown,pct_14_in_state,pct_14_out_of_state,pct_14_unknown,pct_10_in_state,pct_10_out_of_state,pct_10_unknown
0,Democratic,"694,938,481.84","100,883,207.06","10,117,328.46","805,939,017.36","438,568,693.79","74,661,507.08","6,296,267.27","519,526,468.14","511,599,401.37","63,773,103.10","1,670,811.78","577,043,316.25",0.86,0.13,0.01,0.84,0.14,0.01,0.89,0.11,0.00
1,Nonpartisan,"1,642,246.38","336,165.87","-2,307.58","1,976,104.67","1,781,565.01","178,003.05","4,068.00","1,963,636.06","994,909.24","142,553.70","-19,438.72","1,118,024.22",0.83,0.17,-0.00,0.91,0.09,0.00,0.89,0.13,-0.02
2,Republican,"773,921,813.64","71,364,530.01","9,472,896.23","854,759,239.88","498,905,779.94","78,945,614.02","11,198,512.65","589,049,906.61","701,750,841.97","57,352,634.70","875,940.46","759,979,417.13",0.91,0.08,0.01,0.85,0.13,0.02,0.92,0.08,0.00
3,Third-Party,"3,309,432.13","339,879.37","486,249.28","4,135,560.78","12,669,293.15","627,085.94","40,286.04","13,336,665.13","6,863,698.57","852,177.89","1,538,978.63","9,254,855.09",0.80,0.08,0.12,0.95,0.05,0.00,0.74,0.09,0.17


## Calculate out-of-state contributions by party and year for the complete cycle

Group by year and party and in-vs.-out-of-state contribution status and sum contributions.

In [12]:
contributions_by_party_full_cycles = contributions_full_cycles.groupby(["year", "party", "in_out_state"])["amount"].sum().reset_index()
contributions_by_party_full_cycles.head()

,year,party,in_out_state,amount
0,2010,Democratic,in-state,"782,601,351.14"
1,2010,Democratic,out-of-state,"98,978,507.93"
2,2010,Democratic,unknown,"5,873,945.04"
3,2010,Nonpartisan,in-state,"1,393,839.63"
4,2010,Nonpartisan,out-of-state,"197,886.63"


Pivot dataframe to aggregate each party's data in a single row.

In [13]:
contributions_by_party_full_cycles = pd.pivot_table(contributions_by_party_full_cycles, index=["party"], columns=["year", "in_out_state"]).reset_index()
contributions_by_party_full_cycles

party           amount                             \
year                                  2010                              
in_out_state                      in-state  out-of-state      unknown   
0              Democratic   782,601,351.14 98,978,507.93 5,873,945.04   
1             Nonpartisan     1,393,839.63    197,886.63   -16,406.72   
2              Republican 1,050,695,215.16 98,930,400.21 3,278,640.99   
3             Third-Party    16,751,364.84  1,554,587.31 5,214,417.52   

                                                                         \
year                   2014                                        2018   
in_out_state       in-state   out-of-state       unknown       in-state   
0            688,634,127.81 116,003,523.41  9,128,383.19 707,990,241.87   
1              3,559,696.98     464,716.75     10,268.00   1,642,246.38   
2            865,413,734.07 124,561,145.90 14,348,298.86 795,176,405.03   
3             19,055,899.25   1,099,466.87     83,266.73   3,344,964.24   

                                           
year                                       
in_out_state   out-of-state       unknown  
0            104,940,627.95 10,618,189.37  
1                336,165.87     -2,307.58  
2             75,738,193.34 11,253,754.55  
3                342,789.37    486,816.42

Some records have no contributions for certain categories. Let's set those values equal to zero to be sure any calculations we run on them are correct.

In [14]:
contributions_by_party_full_cycles.fillna(0, inplace=True)

Flatten the resulting dataframe's multi-index columns.

In [15]:
contributions_by_party_full_cycles.columns = ["party", "10_in_state", "10_out_of_state", "10_unknown",
                                  "14_in_state", "14_out_of_state", "14_unknown",
                                  "18_in_state", "18_out_of_state", "18_unknown"
                                  ]
contributions_by_party_full_cycles

,party,10_in_state,10_out_of_state,10_unknown,14_in_state,14_out_of_state,14_unknown,18_in_state,18_out_of_state,18_unknown
0,Democratic,"782,601,351.14","98,978,507.93","5,873,945.04","688,634,127.81","116,003,523.41","9,128,383.19","707,990,241.87","104,940,627.95","10,618,189.37"
1,Nonpartisan,"1,393,839.63","197,886.63","-16,406.72","3,559,696.98","464,716.75","10,268.00","1,642,246.38","336,165.87","-2,307.58"
2,Republican,"1,050,695,215.16","98,930,400.21","3,278,640.99","865,413,734.07","124,561,145.90","14,348,298.86","795,176,405.03","75,738,193.34","11,253,754.55"
3,Third-Party,"16,751,364.84","1,554,587.31","5,214,417.52","19,055,899.25","1,099,466.87","83,266.73","3,344,964.24","342,789.37","486,816.42"


Calculate the total contributions by cycle.

In [16]:
contributions_by_party_full_cycles["18_total"] = contributions_by_party_full_cycles["18_in_state"] + contributions_by_party_full_cycles["18_out_of_state"] + contributions_by_party_full_cycles["18_unknown"]
contributions_by_party_full_cycles["14_total"] = contributions_by_party_full_cycles["14_in_state"] + contributions_by_party_full_cycles["14_out_of_state"] + contributions_by_party_full_cycles["14_unknown"]
contributions_by_party_full_cycles["10_total"] = contributions_by_party_full_cycles["10_in_state"] + contributions_by_party_full_cycles["10_out_of_state"] + contributions_by_party_full_cycles["10_unknown"]
contributions_by_party_full_cycles = contributions_by_party_full_cycles[["party", "18_in_state", "18_out_of_state", "18_unknown", "18_total", "14_in_state", "14_out_of_state", "14_unknown", "14_total", "10_in_state", "10_out_of_state", "10_unknown", "10_total"]]

Calculate the proportion of in-state, out-of-state and unknown contributions by cycle.

In [17]:
contributions_by_party_full_cycles["pct_18_in_state"] = contributions_by_party_full_cycles["18_in_state"] / (contributions_by_party_full_cycles["18_in_state"] + contributions_by_party_full_cycles["18_out_of_state"] + contributions_by_party_full_cycles["18_unknown"])
contributions_by_party_full_cycles["pct_18_out_of_state"] = contributions_by_party_full_cycles["18_out_of_state"] / (contributions_by_party_full_cycles["18_in_state"] + contributions_by_party_full_cycles["18_out_of_state"] + contributions_by_party_full_cycles["18_unknown"])
contributions_by_party_full_cycles["pct_18_unknown"] = contributions_by_party_full_cycles["18_unknown"] / (contributions_by_party_full_cycles["18_in_state"] + contributions_by_party_full_cycles["18_out_of_state"] + contributions_by_party_full_cycles["18_unknown"])
contributions_by_party_full_cycles["pct_14_in_state"] = contributions_by_party_full_cycles["14_in_state"] / (contributions_by_party_full_cycles["14_in_state"] + contributions_by_party_full_cycles["14_out_of_state"] + contributions_by_party_full_cycles["14_unknown"])
contributions_by_party_full_cycles["pct_14_out_of_state"] = contributions_by_party_full_cycles["14_out_of_state"] / (contributions_by_party_full_cycles["14_in_state"] + contributions_by_party_full_cycles["14_out_of_state"] + contributions_by_party_full_cycles["14_unknown"])
contributions_by_party_full_cycles["pct_14_unknown"] = contributions_by_party_full_cycles["14_unknown"] / (contributions_by_party_full_cycles["14_in_state"] + contributions_by_party_full_cycles["14_out_of_state"] + contributions_by_party_full_cycles["14_unknown"])
contributions_by_party_full_cycles["pct_10_in_state"] = contributions_by_party_full_cycles["10_in_state"] / (contributions_by_party_full_cycles["10_in_state"] + contributions_by_party_full_cycles["10_out_of_state"] + contributions_by_party_full_cycles["10_unknown"])
contributions_by_party_full_cycles["pct_10_out_of_state"] = contributions_by_party_full_cycles["10_out_of_state"] / (contributions_by_party_full_cycles["10_in_state"] + contributions_by_party_full_cycles["10_out_of_state"] + contributions_by_party_full_cycles["10_unknown"])
contributions_by_party_full_cycles["pct_10_unknown"] = contributions_by_party_full_cycles["10_unknown"] / (contributions_by_party_full_cycles["10_in_state"] + contributions_by_party_full_cycles["10_out_of_state"] + contributions_by_party_full_cycles["10_unknown"])
contributions_by_party_full_cycles

/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

,party,18_in_state,18_out_of_state,18_unknown,18_total,14_in_state,14_out_of_state,14_unknown,14_total,10_in_state,10_out_of_state,10_unknown,10_total,pct_18_in_state,pct_18_out_of_state,pct_18_unknown,pct_14_in_state,pct_14_out_of_state,pct_14_unknown,pct_10_in_state,pct_10_out_of_state,pct_10_unknown
0,Democratic,"707,990,241.87","104,940,627.95","10,618,189.37","823,549,059.19","688,634,127.81","116,003,523.41","9,128,383.19","813,766,034.41","782,601,351.14","98,978,507.93","5,873,945.04","887,453,804.11",0.86,0.13,0.01,0.85,0.14,0.01,0.88,0.11,0.01
1,Nonpartisan,"1,642,246.38","336,165.87","-2,307.58","1,976,104.67","3,559,696.98","464,716.75","10,268.00","4,034,681.73","1,393,839.63","197,886.63","-16,406.72","1,575,319.54",0.83,0.17,-0.00,0.88,0.12,0.00,0.88,0.13,-0.01
2,Republican,"795,176,405.03","75,738,193.34","11,253,754.55","882,168,352.92","865,413,734.07","124,561,145.90","14,348,298.86","1,004,323,178.83","1,050,695,215.16","98,930,400.21","3,278,640.99","1,152,904,256.36",0.90,0.09,0.01,0.86,0.12,0.01,0.91,0.09,0.00
3,Third-Party,"3,344,964.24","342,789.37","486,816.42","4,174,570.03","19,055,899.25","1,099,466.87","83,266.73","20,238,632.85","16,751,364.84","1,554,587.31","5,214,417.52","23,520,369.67",0.80,0.08,0.12,0.94,0.05,0.00,0.71,0.07,0.22


## Calculate out-of-state contributions by office and year

Group by year, standardized office and in-vs.-out-of-state contribution status and sum contributions.

In [18]:
contributions_by_office = contributions.groupby(["year", "standardized_office", "in_out_state"])["amount"].sum().reset_index()
contributions_by_office.head()

,year,standardized_office,in_out_state,amount
0,2010,GOVERNOR/LIEUTENANT GOVERNOR,in-state,"723,096,242.03"
1,2010,GOVERNOR/LIEUTENANT GOVERNOR,out-of-state,"74,493,761.73"
2,2010,GOVERNOR/LIEUTENANT GOVERNOR,unknown,"2,145,307.41"
3,2010,STATE HOUSE/ASSEMBLY/SENATE,in-state,"498,112,609.12"
4,2010,STATE HOUSE/ASSEMBLY/SENATE,out-of-state,"47,626,707.66"


Pivot the dataframe to aggregate each office's data in a single row.

In [19]:
contributions_by_office = pd.pivot_table(contributions_by_office, index=["standardized_office"], columns=["year", "in_out_state"]).reset_index()
contributions_by_office

standardized_office         amount                \
year                                                 2010                 
in_out_state                                     in-state  out-of-state   
0             GOVERNOR/LIEUTENANT GOVERNOR 723,096,242.03 74,493,761.73   
1              STATE HOUSE/ASSEMBLY/SENATE 498,112,609.12 47,626,707.66   

                                                                      \
year                                2014                               
in_out_state      unknown       in-state  out-of-state       unknown   
0            2,145,307.41 430,226,582.69 96,121,512.79 10,271,392.27   
1            1,920,984.74 521,698,749.20 58,290,697.30  7,267,741.69   

                                                          
year                   2018                               
in_out_state       in-state   out-of-state       unknown  
0            937,211,096.27 102,323,339.14  7,613,593.59  
1            536,600,877.72  70,600,443.17 12,460,572.80

Flatten the resulting dataframe's multi-index columns.

In [20]:
contributions_by_office.columns = ["standardized_office", "10_in_state", "10_out_of_state", "10_unknown",
                                  "14_in_state", "14_out_of_state", "14_unknown",
                                  "18_in_state", "18_out_of_state", "18_unknown"
                                  ]
contributions_by_office

,standardized_office,10_in_state,10_out_of_state,10_unknown,14_in_state,14_out_of_state,14_unknown,18_in_state,18_out_of_state,18_unknown
0,GOVERNOR/LIEUTENANT GOVERNOR,"723,096,242.03","74,493,761.73","2,145,307.41","430,226,582.69","96,121,512.79","10,271,392.27","937,211,096.27","102,323,339.14","7,613,593.59"
1,STATE HOUSE/ASSEMBLY/SENATE,"498,112,609.12","47,626,707.66","1,920,984.74","521,698,749.20","58,290,697.30","7,267,741.69","536,600,877.72","70,600,443.17","12,460,572.80"


Calculate the total contributions by cycle.

In [21]:
contributions_by_office["18_total"] = contributions_by_office["18_in_state"] + contributions_by_office["18_out_of_state"] + contributions_by_office["18_unknown"]
contributions_by_office["14_total"] = contributions_by_office["14_in_state"] + contributions_by_office["14_out_of_state"] + contributions_by_office["14_unknown"]
contributions_by_office["10_total"] = contributions_by_office["10_in_state"] + contributions_by_office["10_out_of_state"] + contributions_by_office["10_unknown"]
contributions_by_office = contributions_by_office[["standardized_office", "18_in_state", "18_out_of_state", "18_unknown", "18_total", "14_in_state", "14_out_of_state", "14_unknown", "14_total", "10_in_state", "10_out_of_state", "10_unknown", "10_total"]]
contributions_by_office

,standardized_office,18_in_state,18_out_of_state,18_unknown,18_total,14_in_state,14_out_of_state,14_unknown,14_total,10_in_state,10_out_of_state,10_unknown,10_total
0,GOVERNOR/LIEUTENANT GOVERNOR,"937,211,096.27","102,323,339.14","7,613,593.59","1,047,148,029.00","430,226,582.69","96,121,512.79","10,271,392.27","536,619,487.75","723,096,242.03","74,493,761.73","2,145,307.41","799,735,311.17"
1,STATE HOUSE/ASSEMBLY/SENATE,"536,600,877.72","70,600,443.17","12,460,572.80","619,661,893.69","521,698,749.20","58,290,697.30","7,267,741.69","587,257,188.19","498,112,609.12","47,626,707.66","1,920,984.74","547,660,301.52"


Calculate the proportion of in-state, out-of-state and unknown contributions.

In [22]:
contributions_by_office["pct_18_in_state"] = contributions_by_office["18_in_state"] / (contributions_by_office["18_in_state"] + contributions_by_office["18_out_of_state"] + contributions_by_office["18_unknown"])
contributions_by_office["pct_18_out_of_state"] = contributions_by_office["18_out_of_state"] / (contributions_by_office["18_in_state"] + contributions_by_office["18_out_of_state"] + contributions_by_office["18_unknown"])
contributions_by_office["pct_18_unknown"] = contributions_by_office["18_unknown"] / (contributions_by_office["18_in_state"] + contributions_by_office["18_out_of_state"] + contributions_by_office["18_unknown"])
contributions_by_office["pct_14_in_state"] = contributions_by_office["14_in_state"] / (contributions_by_office["14_in_state"] + contributions_by_office["14_out_of_state"] + contributions_by_office["14_unknown"])
contributions_by_office["pct_14_out_of_state"] = contributions_by_office["14_out_of_state"] / (contributions_by_office["14_in_state"] + contributions_by_office["14_out_of_state"] + contributions_by_office["14_unknown"])
contributions_by_office["pct_14_unknown"] = contributions_by_office["14_unknown"] / (contributions_by_office["14_in_state"] + contributions_by_office["14_out_of_state"] + contributions_by_office["14_unknown"])
contributions_by_office["pct_10_in_state"] = contributions_by_office["10_in_state"] / (contributions_by_office["10_in_state"] + contributions_by_office["10_out_of_state"] + contributions_by_office["10_unknown"])
contributions_by_office["pct_10_out_of_state"] = contributions_by_office["10_out_of_state"] / (contributions_by_office["10_in_state"] + contributions_by_office["10_out_of_state"] + contributions_by_office["10_unknown"])
contributions_by_office["pct_10_unknown"] = contributions_by_office["10_unknown"] / (contributions_by_office["10_in_state"] + contributions_by_office["10_out_of_state"] + contributions_by_office["10_unknown"])
contributions_by_office

/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

,standardized_office,18_in_state,18_out_of_state,18_unknown,18_total,14_in_state,14_out_of_state,14_unknown,14_total,10_in_state,10_out_of_state,10_unknown,10_total,pct_18_in_state,pct_18_out_of_state,pct_18_unknown,pct_14_in_state,pct_14_out_of_state,pct_14_unknown,pct_10_in_state,pct_10_out_of_state,pct_10_unknown
0,GOVERNOR/LIEUTENANT GOVERNOR,"937,211,096.27","102,323,339.14","7,613,593.59","1,047,148,029.00","430,226,582.69","96,121,512.79","10,271,392.27","536,619,487.75","723,096,242.03","74,493,761.73","2,145,307.41","799,735,311.17",0.90,0.10,0.01,0.80,0.18,0.02,0.90,0.09,0.00
1,STATE HOUSE/ASSEMBLY/SENATE,"536,600,877.72","70,600,443.17","12,460,572.80","619,661,893.69","521,698,749.20","58,290,697.30","7,267,741.69","587,257,188.19","498,112,609.12","47,626,707.66","1,920,984.74","547,660,301.52",0.87,0.11,0.02,0.89,0.10,0.01,0.91,0.09,0.00


## Calculate 2018 out-of-state contributions by redistricting role

Filter the contributions data to the 2018 cycle.

In [23]:
contributions_18 = contributions[contributions["year"] == 2018]
contributions_18.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2619767 entries, 0 to 2619766
Data columns (total 23 columns):
candidate                 object
candidate_id              int32
year                      int16
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      datetime64[ns]
contributor_street        object
contributor_city          object
contributor_state         object
contributor_zip           object
in_out_state              object
no_veto                   object
office                    object
latest_month              datetime64[ns]
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: datetime64[ns](2), float64(1), int16(1), int32(1), object(18)
memory usage: 454.7+ MB


Group by redistricting role and in-vs.-out-of-state contribution status and sum contributions.

In [24]:
contributions_by_redistricting = contributions.groupby(["redistricting_role", "in_out_state"])["amount"].sum().reset_index()
contributions_by_redistricting

,redistricting_role,in_out_state,amount
0,N,in-state,"234,469,649.22"
1,N,out-of-state,"28,338,244.60"
2,N,unknown,"4,360,639.26"
3,Y,in-state,"1,239,342,324.77"
4,Y,out-of-state,"144,585,537.71"
5,Y,unknown,"15,713,527.13"


Pivot dataframe to aggregate each role's data in a single row.

In [25]:
contributions_by_redistricting = pd.pivot_table(contributions_by_redistricting, index=["redistricting_role"], columns=["in_out_state"]).reset_index()
contributions_by_redistricting

redistricting_role           amount                             
in_out_state                            in-state   out-of-state       unknown
0                             N   234,469,649.22  28,338,244.60  4,360,639.26
1                             Y 1,239,342,324.77 144,585,537.71 15,713,527.13

Flatten the resulting dataframe's multi-index columns.

In [26]:
contributions_by_redistricting.columns = ["redistricting_role", "in_state", "out_of_state", "unknown"]
contributions_by_redistricting

,redistricting_role,in_state,out_of_state,unknown
0,N,"234,469,649.22","28,338,244.60","4,360,639.26"
1,Y,"1,239,342,324.77","144,585,537.71","15,713,527.13"


Calculate the total contributions by redistricting role.

In [27]:
contributions_by_redistricting["total"] = contributions_by_redistricting["in_state"] + contributions_by_redistricting["out_of_state"] + contributions_by_redistricting["unknown"]

Calculate the proportion of in-state, out-of-state and unknown contributions by redistricting role.

In [28]:
contributions_by_redistricting["pct_in_state"] = contributions_by_redistricting["in_state"] / (contributions_by_redistricting["in_state"] + contributions_by_redistricting["out_of_state"] + contributions_by_redistricting["unknown"])
contributions_by_redistricting["pct_out_of_state"] = contributions_by_redistricting["out_of_state"] / (contributions_by_redistricting["in_state"] + contributions_by_redistricting["out_of_state"] + contributions_by_redistricting["unknown"])
contributions_by_redistricting["pct_unknown"] = contributions_by_redistricting["unknown"] / (contributions_by_redistricting["in_state"] + contributions_by_redistricting["out_of_state"] + contributions_by_redistricting["unknown"])
contributions_by_redistricting

,redistricting_role,in_state,out_of_state,unknown,total,pct_in_state,pct_out_of_state,pct_unknown
0,N,"234,469,649.22","28,338,244.60","4,360,639.26","267,168,533.08",0.88,0.11,0.02
1,Y,"1,239,342,324.77","144,585,537.71","15,713,527.13","1,399,641,389.61",0.89,0.10,0.01


## Calculate contributions by candidate status and year

Group by year and candidate status and in-vs.-out-of-state contribution status and sum contributions.

In [29]:
contributions_by_status = contributions.groupby(["year", "standardized_status", "in_out_state"])["amount"].sum().reset_index()
contributions_by_status

,year,standardized_status,in_out_state,amount
0,2010,ADVANCED TO GENERAL,in-state,"973,295,056.50"
1,2010,ADVANCED TO GENERAL,out-of-state,"104,553,554.45"
2,2010,ADVANCED TO GENERAL,unknown,"3,334,419.94"
3,2010,DID NOT ADVANCE,in-state,"247,913,794.65"
4,2010,DID NOT ADVANCE,out-of-state,"17,566,914.94"
5,2010,DID NOT ADVANCE,unknown,"731,872.21"
6,2014,ADVANCED TO GENERAL,in-state,"820,509,652.40"
7,2014,ADVANCED TO GENERAL,out-of-state,"135,234,615.93"
8,2014,ADVANCED TO GENERAL,unknown,"12,254,832.61"
9,2014,DID NOT ADVANCE,in-state,"131,415,679.49"


Pivot dataframe to aggregate each candidate status' data in a single row.

In [30]:
contributions_by_status = pd.pivot_table(contributions_by_status, index=["standardized_status"], columns=["year", "in_out_state"]).reset_index()
contributions_by_status

standardized_status         amount                              \
year                                        2010                               
in_out_state                            in-state   out-of-state      unknown   
0             ADVANCED TO GENERAL 973,295,056.50 104,553,554.45 3,334,419.94   
1                 DID NOT ADVANCE 247,913,794.65  17,566,914.94   731,872.21   

                                                                           \
year                   2014                                          2018   
in_out_state       in-state   out-of-state       unknown         in-state   
0            820,509,652.40 135,234,615.93 12,254,832.61 1,049,219,119.87   
1            131,415,679.49  19,177,594.16  5,284,301.35   424,592,854.12   

                                           
year                                       
in_out_state   out-of-state       unknown  
0            135,163,430.33 14,934,582.70  
1             37,760,351.98  5,139,583.69

Flatten the resulting dataframe's multi-index columns.

In [31]:
contributions_by_status.columns = ["standardized_status", "10_in_state", "10_out_of_state", "10_unknown",
                                   "14_in_state", "14_out_of_state", "14_unknown",
                                   "18_in_state", "18_out_of_state", "18_unknown"
                                  ]
contributions_by_status

,standardized_status,10_in_state,10_out_of_state,10_unknown,14_in_state,14_out_of_state,14_unknown,18_in_state,18_out_of_state,18_unknown
0,ADVANCED TO GENERAL,"973,295,056.50","104,553,554.45","3,334,419.94","820,509,652.40","135,234,615.93","12,254,832.61","1,049,219,119.87","135,163,430.33","14,934,582.70"
1,DID NOT ADVANCE,"247,913,794.65","17,566,914.94","731,872.21","131,415,679.49","19,177,594.16","5,284,301.35","424,592,854.12","37,760,351.98","5,139,583.69"


Calculate the total contributions by cycle.

In [32]:
contributions_by_status["18_total"] = contributions_by_status["18_in_state"] + contributions_by_status["18_out_of_state"] + contributions_by_status["18_unknown"]
contributions_by_status["14_total"] = contributions_by_status["14_in_state"] + contributions_by_status["14_out_of_state"] + contributions_by_status["14_unknown"]
contributions_by_status["10_total"] = contributions_by_status["10_in_state"] + contributions_by_status["10_out_of_state"] + contributions_by_status["10_unknown"]
contributions_by_status = contributions_by_status[["standardized_status", "18_in_state", "18_out_of_state", "18_unknown", "18_total", "14_in_state", "14_out_of_state", "14_unknown", "14_total", "10_in_state", "10_out_of_state", "10_unknown", "10_total"]]

Calculate the proportion of in-state, out-of-state and unknown contributions by cycle.

In [33]:
contributions_by_status["pct_18_in_state"] = contributions_by_status["18_in_state"] / (contributions_by_status["18_in_state"] + contributions_by_status["18_out_of_state"] + contributions_by_status["18_unknown"])
contributions_by_status["pct_18_out_of_state"] = contributions_by_status["18_out_of_state"] / (contributions_by_status["18_in_state"] + contributions_by_status["18_out_of_state"] + contributions_by_status["18_unknown"])
contributions_by_status["pct_18_unknown"] = contributions_by_status["18_unknown"] / (contributions_by_status["18_in_state"] + contributions_by_status["18_out_of_state"] + contributions_by_status["18_unknown"])
contributions_by_status["pct_14_in_state"] = contributions_by_status["14_in_state"] / (contributions_by_status["14_in_state"] + contributions_by_status["14_out_of_state"] + contributions_by_status["14_unknown"])
contributions_by_status["pct_14_out_of_state"] = contributions_by_status["14_out_of_state"] / (contributions_by_status["14_in_state"] + contributions_by_status["14_out_of_state"] + contributions_by_status["14_unknown"])
contributions_by_status["pct_14_unknown"] = contributions_by_status["14_unknown"] / (contributions_by_status["14_in_state"] + contributions_by_status["14_out_of_state"] + contributions_by_status["14_unknown"])
contributions_by_status["pct_10_in_state"] = contributions_by_status["10_in_state"] / (contributions_by_status["10_in_state"] + contributions_by_status["10_out_of_state"] + contributions_by_status["10_unknown"])
contributions_by_status["pct_10_out_of_state"] = contributions_by_status["10_out_of_state"] / (contributions_by_status["10_in_state"] + contributions_by_status["10_out_of_state"] + contributions_by_status["10_unknown"])
contributions_by_status["pct_10_unknown"] = contributions_by_status["10_unknown"] / (contributions_by_status["10_in_state"] + contributions_by_status["10_out_of_state"] + contributions_by_status["10_unknown"])
contributions_by_status

/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

,standardized_status,18_in_state,18_out_of_state,18_unknown,18_total,14_in_state,14_out_of_state,14_unknown,14_total,10_in_state,10_out_of_state,10_unknown,10_total,pct_18_in_state,pct_18_out_of_state,pct_18_unknown,pct_14_in_state,pct_14_out_of_state,pct_14_unknown,pct_10_in_state,pct_10_out_of_state,pct_10_unknown
0,ADVANCED TO GENERAL,"1,049,219,119.87","135,163,430.33","14,934,582.70","1,199,317,132.90","820,509,652.40","135,234,615.93","12,254,832.61","967,999,100.94","973,295,056.50","104,553,554.45","3,334,419.94","1,081,183,030.89",0.87,0.11,0.01,0.85,0.14,0.01,0.90,0.10,0.00
1,DID NOT ADVANCE,"424,592,854.12","37,760,351.98","5,139,583.69","467,492,789.79","131,415,679.49","19,177,594.16","5,284,301.35","155,877,575.00","247,913,794.65","17,566,914.94","731,872.21","266,212,581.80",0.91,0.08,0.01,0.84,0.12,0.03,0.93,0.07,0.00


## Export the data

In [34]:
%%notify
writer = pd.ExcelWriter("data/national_analysis.xlsx")
contributions_by_party_full_cycles.to_excel(writer, "contributions_party_full_cycles", index=False)
contributions_by_party.to_excel(writer, "contributions_by_party", index=False)
contributions_by_office.to_excel(writer, "contributions_by_office", index=False)
contributions_by_redistricting.to_excel(writer, "contributions_by_redistricting", index=False)
contributions_by_status.to_excel(writer, "contributions_by_status", index=False)
writer.save()

<IPython.core.display.Javascript object>